In [1]:
import TableParser
import pandas as pd
import tempfile, zipfile
import os

In [2]:
class TestTableParser(TableParser.TableParserItem):
    def __init__(self):
        pass
    
    def parse(self, series: pd.Series):
        mistake = {
            'Jan':'01','Feb': '02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sept':'09','Oct':'10','Nov':'11','Dec':'12',
            'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5',
            r'[ -/]|[:-@]|[\[-_]|[{-~]': '-',
        }
        for key, val in mistake.items():
            series = series.str.strip().str.replace(key, val, regex=True)
        return (pd.to_datetime(series, errors="ignore"), [])

    def fix_xlsx(self, in_file):
        tmpfd, tmp = tempfile.mkstemp(dir=os.path.dirname(in_file))
        os.close(tmpfd)
        filename = '[Content_Types].xml'
        data = ''
        with zipfile.ZipFile(in_file, 'r') as zin:
            with zipfile.ZipFile(tmp, 'w') as zout:
                for item in zin.infolist():
                    if item.filename != filename:
                        zout.writestr(item, zin.read(item.filename))
                    else:
                        data = zin.read(filename).decode()
        os.remove(in_file)
        os.rename(tmp, in_file)
        data = data.replace('/xl/sharedStrings.xml', '/xl/SharedStrings.xml')
        with zipfile.ZipFile(in_file, mode='a', compression=zipfile.ZIP_DEFLATED) as zf:
            zf.writestr(filename, data)

    ###############################################
    # to find the x,y coordinate of row with date #
    
    def find_coordinate(self, x, y,df):
        test_list =[[0,0]]
        count_X_axis,count_Y_axis,main_axis = 0,0,"Y axis"
        
        for i in range(1,df.shape[0]):
            if len(test_list) == 11: 
                break
            for j in range(df.shape[1]):
                if type(df.iloc[i,j]) == type("string"):
                    if type(testTableParser.parse(pd.Series([df.iloc[i,j]]))[0][0]) == type(pd.to_datetime('01.01.2021')):
                        test_list.append([i,j])
                        if test_list[-1][1] > test_list[-2][1]:
                            count_X_axis += 1
                        if test_list[-1][0] > test_list[-2][0]:
                            count_Y_axis += 1  
                        if len(test_list) == 11:
                            break
        if count_X_axis > count_Y_axis:
            main_axis = "X axis"
#         print(main_axis,test_list[1])
        return test_list[1][0],test_list[1][1],main_axis

    def all_action(self,df):
        data_row_num, data_col_num,main_axis = self.find_coordinate(0,0,df) # <-- find coordinate of dates
        if main_axis == "X axis":   
            date = df.iloc[data_row_num, data_col_num:].reset_index(drop=True)  # <--- Series with date (01.03.2016;...)
            table = df.iloc[data_row_num:, data_col_num:].reset_index(drop=True)  # <--- all useful data
        elif main_axis == "Y axis":
            pass
        date = self.parse(date) # < --- parse date for correction
        table.iloc[0] = date[0] # < --- set new call date to the table
#         print(table.iloc[0].reset_index(drop=True))         # check of new date in data  ##(((JUST switch ON/OFF)))##
#         table.columns = table.iloc[0] # <--- change column names
#         table = table.iloc[1:, :] # <--- remove first row
        
        return table

In [5]:
testTableParser = TestTableParser()
testTableParser.fix_xlsx('file3.xlsx')
df = pd.read_excel('file3.xlsx')
testTableParser.all_action(df)

KeyError: "There is no item named 'xl/SharedStrings.xml' in the archive"

In [ ]:
testTableParser.parse(pd.Series(['01.02.2002']))[0][0]

In [ ]:
# pd.to_datetime('01.02.2002')
df.shape